In [16]:
# import splinter and BSoup
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as Soup
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# setup executable path 

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/alfred/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


In [3]:
# Visit the mars nasa news site

url = 'https://redplanetscience.com'
browser.visit(url)
#Optional delay for loading the page
# searching for elements with a specific combo of tag (div) and attribute (list_text)
# as example ul.item_list would be found in HTML as <ul class="item_list">
# also tellilng the browser to wait one second before searching for components, is useful because sometimes
# dynamic pages take a little while to load especially if they're image heavy
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [5]:
# setup the HTML parser

html = browser.html
news_soup = Soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

# slide_elem is the variable to look for the <div /> tag and its decendents, this is our parent element
# we'll reference it when we want to filter search results within it and we'll reference it when we want to 
# filter serach results even further
# the '.' is used for selecting classes, such as 'list_text' so the code 'div.list_text' pinpoints the <div />
# tag with the class of list_text
# CSS works from right to left, such as returning the last item on the list instead of the first
# because of this when using select_one the first matching element reutrned will be a <li /> element
# witha a class of slide and all nested elements within it

In [6]:
# we want to assign the title and summary text to variables
# begin scraping

slide_elem.find('div', class_='content_title')

<div class="content_title">NASA's Curiosity Mars Rover Takes a New Selfie Before Record Climb</div>

In [7]:
# Use the parent element to find the first 'a' tag and save it as 'news-title'

news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"NASA's Curiosity Mars Rover Takes a New Selfie Before Record Climb"

In [8]:
# use the parent element to find the paragraph text

news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'Along with capturing an image before its steepest ascent ever, the robotic explorer filmed its "selfie stick," or robotic arm, in action.'

### Featured Images

In [10]:
# visit url

url = 'https://spaceimages-mars.com'
browser.visit(url)

In [11]:
# find and click the full size image button
# with the index we tell lthe browser to click the second button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [13]:
# parse the resulting html with soup

html = browser.html
img_soup = Soup(html, 'html.parser')

In [14]:
# find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

# an img tag is nested within this HTML so we've included it
# .get(src) pulls the link to the image

'image/featured/mars2.jpg'

In [15]:
# use the base URL to create an absolute URL

img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

In [17]:
# get the data from the mars facts site from the table in the form of a data frame

# createing a new DF from the HTML table read_html() specifically searches for and returns a list of tables
# found in the HTML, by specifying an index of 0 we're telling pandas to pull only the first table it encounters
# it then turns the table into a DF
df = pd.read_html('https://galaxyfacts-mars.com')[0]

# we assign columns to the new DF for additional clarity
df.columns=['description', 'Mars', 'Earth']

#by using the .set_index() function we're turning the description column ino the DF's index
# inplace=True means that the updated index will remain in place without having to reassign the DF to a new 
# variable
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [18]:
# convert the df to html

df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [19]:
# end the automated browsing session
browser.quit()